In [ ]:
import pandas as pd
from sklearn
EX = pd.read_csv("https://raw.githubusercontent.com/soules-one/ML/refs/heads/main/lab1/ex.csv)
TEST = pd.read_csv("https://raw.githubusercontent.com/soules-one/ML/refs/heads/main/lab1/test.csv)
TRAIN = pd.read_csv("https://raw.githubusercontent.com/soules-one/ML/refs/heads/main/lab1/train.csv)



LAB1